In [ ]:
# This code is retrieved from Michel's notebook
# READ: https://hatchjs.com/graph-from-place-osmnx/

In [1]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
import osmnx as ox # for handling data from OpenStreetMap (osm) with the help of networkX (nx)
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations
from shapely.geometry import Polygon
import numpy as np
import pandana as pdna
import os
from tqdm import tqdm

In [2]:
# RUN ONLY ONCE TO SAVE THE GRAPHS - ALREADY DONE

# walking_graphs = {}

# # specify parameters for the analysis
# walk_time = 15  # max walking horizon in minutes
# walk_speed = 5  # km per hour
# walk_time_sec = walk_time * 60 # We need the time in seconds to match how travel time is calculated in OSMnx

# # Loop through the neighborhoods
# for neighborhood in tqdm(neighbourhoods):
#     # Create a graph for the current neighborhood
#     G_walk_neighborhood = ox.graph_from_place(neighborhood, network_type='walk') # type of transportation is set here

#     for u, v, data in G_walk_neighborhood.edges(data=True):
#         data['speed_kph'] = walk_speed
#     G_walk_neighborhood = ox.add_edge_travel_times(G_walk_neighborhood) # this computes the travel time in seconds based on the 'speed_kph' column

# # caro path
#     # ox.save_graphml(G_walk_neighborhood, f"/Users/caro/Desktop/SPRING24/GDS/PROJECT/graphs/G_walk_{neighborhood}.graphml") # save graph per neighbourhood
# # michel path
#     ox.save_graphml(G_walk_neighborhood, f"../graphs/G_walk_{neighborhood}.graphml") # save graph per neighbourhood
#     # Store the graph in the dictionary using the neighborhood name as the key
#     walking_graphs[neighborhood] = G_walk_neighborhood

In [3]:
place = 'Montreal, Canada'
# list_of_amenities = ['restaurant', 'cafe', 'pharmacy', 'hospital', 'parking']
list_of_amenities = [
    "restaurant",
    "post_box",
    "pub",
    "atm",
    "townhall",
    "post_office",
    "kindergarten",
    "cinema",
    "cafe",
    "fire_station",
    "bank",
    "school",
    "place_of_worship",
    "childcare",
    "police",
    "pharmacy",
    "fast_food",
    "library",
    "clinic",
    "bar",
    "dentist",
    "theatre",
    "ice_cream",
    "doctors",
    "community_centre",
    "bureau_de_change",
    "veterinary",
    "arts_centre",
    "studio",
    "nightclub",
    "recycling",
    "marketplace",
    "college",
    "social_facility",
    "hospital",
    "social_centre",
    "university",
    "events_venue",
    "public_building"
]
tags = {'amenity': list_of_amenities}

walk_time = 15  # max walking horizon in minutes
walk_speed = 5  # km per hour
walk_time_sec = walk_time * 60 # We need the time in seconds to match how travel time is calculated in OSMnx

bike_time = 15  # max walking horizon in minutes
bike_speed = 15  # km per hour
bike_time_sec = bike_time * 60 # We need the time in seconds to match how travel time is calculated in OSMnx

drive_time = 15
drive_speed = 50
drive_time_sec = drive_time * 60


In [4]:
# DRIVING

amenities = ox.features_from_place(place, tags=tags)
amenities = amenities.to_crs('EPSG:25832')

# Dictionary to store centroids for each amenity
centroids_per_amenity = {}

# Loop through each amenity category
for amenity in list_of_amenities:
    # Filter amenities for the current category
    amenities_category = amenities[amenities['amenity'] == amenity]
    # Calculate centroids for the current category
    centroids_category = amenities_category.centroid
    # Store centroids for the current category in the dictionary
    centroids_per_amenity[amenity] = centroids_category

# graphs_dir = '/Users/caro/Desktop/SPRING24/GDS/PROJECT/GDS_project/graphs/drive'
graphs_dir = '../graphs/drive'

driving_graphs = {}
for file_name in tqdm(os.listdir(graphs_dir)):
    if file_name.endswith(".graphml"):
        neighborhood = file_name.replace("G_drive_", "").replace(".graphml", "")
        file_path = os.path.join(graphs_dir, file_name)
        G_drive_neighborhood = ox.load_graphml(file_path)
        driving_graphs[neighborhood] = G_drive_neighborhood
            

drive_pandanas = {}
# Build Pandana network for each neighborhood
for neighborhood, graph in tqdm(driving_graphs.items()):
    graph = ox.project_graph(graph, to_crs='EPSG:25832')
    nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
    edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]
    
    network = pdna.Network(node_x=nodes['x'],
                            node_y=nodes['y'], 
                            edge_from=edges['u'],
                            edge_to=edges['v'],
                            edge_weights=edges[['travel_time']])
    
    drive_pandanas[neighborhood] = network


driving_distances = {}  # Initialize an empty dictionary to store distances for each amenity

for amenity in list_of_amenities:
    driving_distances[amenity] = {}

    for neighborhood, pandana in tqdm(drive_pandanas.items()):
        # Set points of interest (POIs) for the current amenity in the current neighborhood
        pandana.set_pois(category=amenity,  # Set the current amenity category dynamically
                        maxdist=drive_time_sec,
                        maxitems=3,
                        x_col=centroids_per_amenity[amenity].x,  # Use the centroid of the current amenity
                        y_col=centroids_per_amenity[amenity].y)
        
        # Find the nearest POIs for the current amenity in the current neighborhood
        distances = pandana.nearest_pois(distance=drive_time_sec,
                                        category=amenity,  # Set the current amenity category dynamically
                                        num_pois=3)
        
        # Convert travel time from seconds to minutes
        distances['travel_time'] = distances[1] / 60
        
        # Store the distances for the current amenity in the current neighborhood
        driving_distances[amenity][neighborhood] = distances

 11%|█         | 2/18 [00:00<00:03,  5.05it/s]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 1589
Setting CH edge vector of size 3865
Range graph removed 2722 edges of 7730
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 946
Setting CH edge vector of size 2338
Range graph removed 1526 edges of 4676
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 663
Setting CH edge vector of size 1496
Range graph removed 762 edges of 2992
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

 22%|██▏       | 4/18 [00:02<00:08,  1.63it/s]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 1938
Setting CH edge vector of size 5262
Range graph removed 4600 edges of 10524
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 297
Setting CH edge vector of size 767
Range graph removed 554 edges of 1534
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

 33%|███▎      | 6/18 [00:02<00:06,  1.78it/s]

...

 89%|████████▉ | 16/18 [00:08<00:00,  3.21it/s]

Setting CH node vector of size 1744
Setting CH edge vector of size 4113
Range graph removed 2838 edges of 8226
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 1181
Setting CH edge vector of size 2743
Range graph removed 1722 edges of 5486
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

100%|██████████| 18/18 [00:09<00:00,  1.99it/s]


Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 242
Setting CH edge vector of size 620
Range graph removed 502 edges of 1240
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 713
Setting CH edge vector of size 1793
Range graph removed 1478 edges of 3586
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

100%|██████████| 18/18 [00:00<00:00, 106.49it/s]


In [5]:
dfs = []

# Iterate through each amenity and its corresponding DataFrame
for amenity, neighborhood_df_dict in driving_distances.items(): # CHANGE HERE
    # Iterate through each neighborhood DataFrame
    for neighborhood, df in neighborhood_df_dict.items():
        # Add a column for 'amenity' and 'neighborhood' to the DataFrame
        df['amenity'] = amenity
        df['neighborhood'] = neighborhood
        
        # Append the modified DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list along the rows axis
combined_df = pd.concat(dfs, ignore_index=True)

# Assuming your DataFrame is named combined_df
combined_df.to_parquet('../distances/driving_distances.parquet', index=False)  # Set index=False if you don't want to save the index. CHANGE HERE

/home/micp/.local/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [4]:
# WALKING

amenities = ox.features_from_place(place, tags=tags)
amenities = amenities.to_crs('EPSG:25832')

# Dictionary to store centroids for each amenity
centroids_per_amenity = {}

# Loop through each amenity category
for amenity in list_of_amenities:
    # Filter amenities for the current category
    amenities_category = amenities[amenities['amenity'] == amenity]
    # Calculate centroids for the current category
    centroids_category = amenities_category.centroid
    # Store centroids for the current category in the dictionary
    centroids_per_amenity[amenity] = centroids_category

# graphs_dir = '/Users/caro/Desktop/SPRING24/GDS/PROJECT/GDS_project/graphs/walk'
graphs_dir = '../graphs/walk'

walking_graphs = {}
for file_name in tqdm(os.listdir(graphs_dir)):
    if file_name.endswith(".graphml"):
        neighborhood = file_name.replace("G_walk_", "").replace(".graphml", "")
        file_path = os.path.join(graphs_dir, file_name)
        G_walk_neighborhood = ox.load_graphml(file_path)
        walking_graphs[neighborhood] = G_walk_neighborhood
            

walk_pandanas = {}
# Build Pandana network for each neighborhood
for neighborhood, graph in tqdm(walking_graphs.items()):
    graph = ox.project_graph(graph, to_crs='EPSG:25832')
    nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
    edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]
    
    network = pdna.Network(node_x=nodes['x'],
                            node_y=nodes['y'], 
                            edge_from=edges['u'],
                            edge_to=edges['v'],
                            edge_weights=edges[['travel_time']])
    
    walk_pandanas[neighborhood] = network


walking_distances = {}  # Initialize an empty dictionary to store distances for each amenity

for amenity in list_of_amenities:
    walking_distances[amenity] = {}

    for neighborhood, pandana in tqdm(walk_pandanas.items()):
        # Set points of interest (POIs) for the current amenity in the current neighborhood
        pandana.set_pois(category=amenity,  # Set the current amenity category dynamically
                        maxdist=walk_time_sec,
                        maxitems=3,
                        x_col=centroids_per_amenity[amenity].x,  # Use the centroid of the current amenity
                        y_col=centroids_per_amenity[amenity].y)
        
        # Find the nearest POIs for the current amenity in the current neighborhood
        distances = pandana.nearest_pois(distance=walk_time_sec,
                                        category=amenity,  # Set the current amenity category dynamically
                                        num_pois=3)
        
        # Convert travel time from seconds to minutes
        distances['travel_time'] = distances[1] / 60
        
        # Store the distances for the current amenity in the current neighborhood
        walking_distances[amenity][neighborhood] = distances
        
dfs = []

# Iterate through each amenity and its corresponding DataFrame
for amenity, neighborhood_df_dict in walking_distances.items(): # CHANGE HERE
    # Iterate through each neighborhood DataFrame
    for neighborhood, df in neighborhood_df_dict.items():
        # Add a column for 'amenity' and 'neighborhood' to the DataFrame
        df['amenity'] = amenity
        df['neighborhood'] = neighborhood
        
        # Append the modified DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list along the rows axis
combined_df = pd.concat(dfs, ignore_index=True)

# Assuming your DataFrame is named combined_df
combined_df.to_parquet('../distances/walking_distances.parquet', index=False)  # Set index=False if you don't want to save the index. CHANGE HERE

  6%|▌         | 1/18 [00:04<01:09,  4.06s/it]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 3382
Setting CH edge vector of size 10774
Range graph removed 10912 edges of 21548
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


 11%|█         | 2/18 [00:14<02:02,  7.66s/it]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 8890
Setting CH edge vector of size 29956
Range graph removed 30224 edges of 59912
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


 17%|█▋        | 3/18 [00:27<02:29,  9.99s/it]

...

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 3761
Setting CH edge vector of size 11500
Range graph removed 11874 edges of 23000
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

100%|██████████| 18/18 [01:05<00:00,  3.61s/it]


Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 16731
Setting CH edge vector of size 55202
Range graph removed 55584 edges of 110404
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


100%|██████████| 18/18 [00:01<00:00, 12.28it/s]
/home/micp/.local/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


: 

In [6]:
dfs = []

# Iterate through each amenity and its corresponding DataFrame
for amenity, neighborhood_df_dict in walking_distances.items(): # CHANGE HERE
    # Iterate through each neighborhood DataFrame
    for neighborhood, df in neighborhood_df_dict.items():
        # Add a column for 'amenity' and 'neighborhood' to the DataFrame
        df['amenity'] = amenity
        df['neighborhood'] = neighborhood
        
        # Append the modified DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list along the rows axis
combined_df = pd.concat(dfs, ignore_index=True)

# Assuming your DataFrame is named combined_df
combined_df.to_parquet('../distances/walking_distances.parquet', index=False)  # Set index=False if you don't want to save the index. CHANGE HERE

: 

In [11]:
# BIKING

amenities = ox.features_from_place(place, tags=tags)
amenities = amenities.to_crs('EPSG:25832')

# Dictionary to store centroids for each amenity
centroids_per_amenity = {}

# Loop through each amenity category
for amenity in list_of_amenities:
    # Filter amenities for the current category
    amenities_category = amenities[amenities['amenity'] == amenity]
    # Calculate centroids for the current category
    centroids_category = amenities_category.centroid
    # Store centroids for the current category in the dictionary
    centroids_per_amenity[amenity] = centroids_category

biking_graphs = {}
# graphs_dir = '/Users/caro/Desktop/SPRING24/GDS/PROJECT/GDS_project/graphs/bike'
graphs_dir = '../graphs/bike'

if biking_graphs == {}:
    for file_name in tqdm(os.listdir(graphs_dir)):
        if file_name.endswith(".graphml"):
            neighborhood = file_name.replace("G_bike_", "").replace(".graphml", "")
            file_path = os.path.join(graphs_dir, file_name)
            G_bike_neighborhood = ox.load_graphml(file_path)
            biking_graphs[neighborhood] = G_bike_neighborhood
            

bike_pandanas = {}
# Build Pandana network for each neighborhood
for neighborhood, graph in tqdm(biking_graphs.items()):
    graph = ox.project_graph(graph, to_crs='EPSG:25832')
    nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
    edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]
    
    network = pdna.Network(node_x=nodes['x'],
                            node_y=nodes['y'], 
                            edge_from=edges['u'],
                            edge_to=edges['v'],
                            edge_weights=edges[['travel_time']])
    
    bike_pandanas[neighborhood] = network


biking_distances = {}  # Initialize an empty dictionary to store distances for each amenity

for amenity in list_of_amenities:
    biking_distances[amenity] = {}

    for neighborhood, pandana in tqdm(bike_pandanas.items()):
        # Set points of interest (POIs) for the current amenity in the current neighborhood
        pandana.set_pois(category=amenity,  # Set the current amenity category dynamically
                        maxdist=bike_time_sec,
                        maxitems=3,
                        x_col=centroids_per_amenity[amenity].x,  # Use the centroid of the current amenity
                        y_col=centroids_per_amenity[amenity].y)
        
        # Find the nearest POIs for the current amenity in the current neighborhood
        distances = pandana.nearest_pois(distance=bike_time_sec,
                                        category=amenity,  # Set the current amenity category dynamically
                                        num_pois=3)
        
        # Convert travel time from seconds to minutes
        distances['travel_time'] = distances[1] / 60
        
        # Store the distances for the current amenity in the current neighborhood
        biking_distances[amenity][neighborhood] = distances


  6%|▌         | 1/18 [00:00<00:16,  1.05it/s]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 3035
Setting CH edge vector of size 8320
Range graph removed 7628 edges of 16640
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

 11%|█         | 2/18 [00:01<00:10,  1.48it/s]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 2740
Setting CH edge vector of size 7361
Range graph removed 6826 edges of 14722
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

 17%|█▋        | 3/18 [00:01<00:07,  1.95it/s]

...

 94%|█████████▍| 17/18 [00:11<00:00,  2.05it/s]

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 1256
Setting CH edge vector of size 3212
Range graph removed 2388 edges of 6424
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

100%|██████████| 18/18 [00:12<00:00,  1.44it/s]


Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 3848
Setting CH edge vector of size 8718
Range graph removed 6660 edges of 17436
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


100%|██████████| 18/18 [00:00<00:00, 19.32it/s]


In [15]:
dfs = []

# Iterate through each amenity and its corresponding DataFrame
for amenity, neighborhood_df_dict in biking_distances.items(): # CHANGE HERE
    # Iterate through each neighborhood DataFrame
    for neighborhood, df in neighborhood_df_dict.items():
        # Add a column for 'amenity' and 'neighborhood' to the DataFrame
        df['amenity'] = amenity
        df['neighborhood'] = neighborhood
        
        # Append the modified DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames in the list along the rows axis
combined_df = pd.concat(dfs, ignore_index=True)

# Assuming your DataFrame is named combined_df
combined_df.to_parquet('../distances/biking_distances.parquet', index=False)  # Set index=False if you don't want to save the index. CHANGE HERE

/home/micp/.local/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [1]:
combined_df['neighborhood'].value_counts()

NameError: name 'combined_df' is not defined

In [ ]:
# function for plotting
def plot_neighborhood_graph(transportation_type, neighbourhood, distances_by_transportation, amenity):
    
    # Load the appropriate graph based on the transportation type
    if transportation_type == "walking":
        G = walking_graphs[f"{neighbourhood}, Montreal, Canada"]
    elif transportation_type == "driving":
        G = driving_graphs[f"{neighbourhood}, Montreal, Canada"]
    elif transportation_type == "biking":
        G = biking_graphs[f"{neighbourhood}, Montreal, Canada"]
    # CRS
    G_proj = ox.project_graph(G)
    
    distances = distances_by_transportation[amenity][f"{neighbourhood}, Montreal, Canada"]
    
    # Plot the graph with a light background
    fig, ax = ox.plot_graph(G_proj, figsize=(10, 8), bgcolor='white', edge_color='#CCCCCC', edge_linewidth=0.5, node_size=0, show=False, close=False)
    
    # Assuming 'nodes_anjou' is a DataFrame containing node positions and 'distances_anjou' contains the data to plot
    nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
    
    # Scatter plot on the same Axes instance
    sc = ax.scatter(x=nodes_proj["x"], y=nodes_proj["y"], c=distances['travel_time'], s=30, cmap='inferno_r', alpha=0.8)
    
    # Add colorbar
    plt.colorbar(sc, ax=ax, shrink=0.7)
    
    # Show the plot
    plt.show()


In [ ]:
# example usage
plot_neighborhood_graph(transportation_type='biking',
                        neighbourhood='Saint-Laurent',
                        distances_by_transportation=biking_distances,
                        amenity='pharmacy')

In [ ]:
plot_neighborhood_graph(transportation_type='biking',
                        neighbourhood='Saint-Laurent', 
                        distances_by_transportation=biking_distances, 
                        amenity='hospital')

In [ ]:
plot_neighborhood_graph(transportation_type='walking',
                        neighbourhood='Saint-Laurent', 
                        distances_by_transportation=walking_distances, 
                        amenity='hospital')

In [ ]:
plot_neighborhood_graph(transportation_type='driving',
                        neighbourhood='Côte-des-Neiges–Notre-Dame-de-Grâce', 
                        distances_by_transportation=driving_distances, 
                        amenity='restaurant')

In [ ]:
plot_neighborhood_graph(transportation_type='driving',
                        neighbourhood='Saint-Laurent', 
                        distances_by_transportation=driving_distances, 
                        amenity='restaurant')

In [ ]:
plot_neighborhood_graph(transportation_type='walking',
                        neighbourhood='Saint-Léonard', 
                        distances_by_transportation=walking_distances, 
                        amenity='hospital')

In [ ]:
plot_neighborhood_graph(transportation_type='walking',
                        neighbourhood='Anjou', 
                        distances_by_transportation=walking_distances, 
                        amenity='restaurant')

In [ ]:
plot_neighborhood_graph(transportation_type='biking',
                        neighbourhood='Anjou', 
                        distances_by_transportation=biking_distances, 
                        amenity='hospital')

In [ ]:
plot_neighborhood_graph(transportation_type='walking',
                        neighbourhood='Le Plateau-Mont-Royal', 
                        distances_by_transportation=walking_distances, 
                        amenity='cafe')